<h2>Лемматизация<h2>

<h4>Лемматизация — это процесс преобразования слова в его базовую форму. Разница между стемминг (stemming) и лемматизацией заключается в том, что лемматизация учитывает контекст и преобразует слово в его значимую базовую форму, тогда как стемминг просто удаляет последние несколько символов, что часто приводит к неверному значению и орфографическим ошибкам.<h4>

<h4>Импортируем библиотеки<h4>

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from collections import Counter
import spacy
from model_selction import model_selection_word_count, model_selection_word_exist, model_selection_tfidf

In [2]:
%ls data

p00_tweets.zip*         processedNeutral.csv*
processedNegative.csv*  processedPositive.csv*


<h4>В качестве примера рассмотрим содержимое файла 'processedNegative.csv' после применения метода<h4>

In [2]:
neg_df = pd.read_csv('data/processedNegative.csv').T.reset_index()
neg_text = " ".join([tweet[0] for tweet in neg_df.values.tolist()])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
neg_lemms = [word.lemma_ for word in nlp(neg_text) if (not word.lemma_ in stopwords.words('english'))]

neg_lemms

['unhappy',
 ' ',
 'dog',
 'like',
 'though',
 'talk',
 'driver',
 'I',
 'goinghe',
 'say',
 'would',
 'love',
 'go',
 'New',
 'York',
 'since',
 'Trump',
 'probably',
 'anybody',
 'know',
 'Rand',
 "'s",
 'likely',
 'fall',
 'dollar',
 '?',
 'I',
 'get',
 'money',
 ' ',
 'I',
 'need',
 'change',
 'r',
 'keep',
 'get',
 'strong',
 'unhappy',
 ' ',
 'I',
 'miss',
 'go',
 'gig',
 'Liverpool',
 'unhappy',
 ' ',
 'new',
 'riverdale',
 'tonight',
 '?',
 'unhappy',
 ' ',
 'A*dy',
 'guy',
 'pop',
 'Asia',
 'translator',
 'prob',
 'go',
 'around',
 'Aus',
 'unhappy',
 ' ',
 'chair',
 'sit',
 '?',
 'I',
 'find',
 '.',
 'everyone',
 'know',
 '.',
 "'ve",
 'shame',
 'I',
 'pu',
 'like',
 'jittery',
 'caffeine',
 'make',
 'I',
 'sad',
 ' ',
 'area',
 'list',
 'unhappy',
 ' ',
 'think',
 'I',
 'go',
 'LibDems',
 'anyway',
 'I',
 'want',
 'fun',
 'plan',
 'weekend',
 'unhappy',
 ' ',
 'notice',
 'I',
 '.',
 ' ',
 'unhappy',
 ' ',
 '?',
 '  ',
 'ahhhhh',
 '!',
 'recognize',
 'LOGAN',
 '!',
 '!',
 '!'

<h4>Функция, которая создасть набор данных для обучения моделей<h4>

In [5]:
def lemma_file_to_df(file_name):
    neg_fn, neut_fn, pos_fn = file_name

    neg_df = pd.read_csv(neg_fn).T.reset_index()
    neut_df = pd.read_csv(neut_fn).T.reset_index()
    pos_df = pd.read_csv(pos_fn).T.reset_index()
    
    neg_text = " ".join([tweet[0] for tweet in neg_df.values.tolist()])
    neut_text = " ".join([tweet[0] for tweet in neut_df.values.tolist()])
    pos_text = " ".join([tweet[0] for tweet in pos_df.values.tolist()])

    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    
    neg_words = Counter([word.lemma_ for word in nlp(neg_text) if (not word.lemma_ in stopwords.words('english'))])
    neut_words = Counter([word.lemma_ for word in nlp(neut_text) if (not word.lemma_ in stopwords.words('english'))])
    pos_words = Counter([word.lemma_ for word in nlp(pos_text) if (not word.lemma_ in stopwords.words('english'))])
    
    unic_words = list(set(neg_words.keys()) | set(neut_words.keys()) | set(pos_words.keys()))

    neg_exist_index = 0
    neut_exist_index = 1
    pos_exist_index = 2
    neg_count_index = 3
    neut_count_index = 4
    pos_count_index = 5
    word_count_index = 6
    neg_tfidf_index = 7
    neut_tfidf_index = 8
    pos_tfidf_index = 9

    df = np.zeros((len(unic_words), 10))
    for i, word in enumerate(unic_words):
        if word in neg_words.keys():
            df[i,neg_exist_index] = 1
            df[i,neg_count_index] = neg_words[word]
        if word in neut_words.keys():
            df[i,neut_exist_index] = 1
            df[i,neut_count_index] = neut_words[word]
        if word in pos_words.keys():
            df[i,pos_exist_index] = 1
            df[i,pos_count_index] = pos_words[word]

    df[:,word_count_index] = df[:,neg_count_index] + df[:,neut_count_index] + df[:,pos_count_index]
    df[:,neg_tfidf_index] = df[:,neg_count_index] / df[:,word_count_index]
    df[:,neut_tfidf_index] = df[:,neut_count_index] / df[:,word_count_index]
    df[:,pos_tfidf_index] = df[:,pos_count_index] / df[:,word_count_index]

    lemma_df = pd.DataFrame(df, columns=[
        'Negative', 'Neutral', 'Positive',
        'Negative counts', 'Neutral counts', 'Positive counts', 'Word counts',
        'Negative TFIDF', 'Neutral TFIDF', 'Positive TFIDF'])
    lemma_df["word"] = unic_words
    return lemma_df, unic_words

<h4>Узнаем, как называются остальные файлы, содержащие исходный набор данных<h4>

In [5]:
%ls data

p00_tweets.zip*         processedNeutral.csv*
processedNegative.csv*  processedPositive.csv*


<h4>Создадим набор данных для обучения<h4>

In [6]:
file_names = ('data/processedNegative.csv', 'data/processedNeutral.csv', 'data/processedPositive.csv')
lemma_df, unic_words = lemma_file_to_df(file_names)
lemma_df

,Negative,Neutral,Positive,Negative counts,Neutral counts,Positive counts,Word counts,Negative TFIDF,Neutral TFIDF,Positive TFIDF,word
0,1.0,0.0,1.0,1.0,0.0,3.0,4.0,0.25,0.000,0.750,store
1,0.0,1.0,0.0,0.0,4.0,0.0,4.0,0.00,1.000,0.000,Datta
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.00,1.000,0.000,pale
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.00,1.000,0.000,.27
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.00,0.000,1.000,Sister
...,...,...,...,...,...,...,...,...,...,...,...
6121,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.00,0.000,1.000,celebrating
6122,1.0,1.0,1.0,4.0,3.0,1.0,8.0,0.50,0.375,0.125,drop
6123,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.00,0.000,0.000,although
6124,1.0,0.0,1.0,1.0,0.0,3.0,4.0,0.25,0.000,0.750,Friday


<h4>Узнаем полученную точность модели<h4>

In [8]:
word_exist_accuracy_score = model_selection_word_exist(lemma_df, unic_words)
word_count_accuracy_score = model_selection_word_count(lemma_df, unic_words)
tfidf_accuracy_score = model_selection_tfidf(lemma_df, unic_words)

/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/val/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: T

In [9]:
print(f"""Accuracy score by word exist: {word_exist_accuracy_score}
Accuracy score by word count: {word_count_accuracy_score}
Fccuracy score by tfidf: {tfidf_accuracy_score}""")

Accuracy score by word exist: 0.48531810766721045
Accuracy score by word count: 0.9445350734094616
Fccuracy score by tfidf: 0.48694942903752036
